Support for 7zip files in Python is limited. Although pandas and other data processing libraries have the ability to directly read gzip and bz2 files, some of the compression algorithms used by 7zip are not well supported by available Python modules. Regardless, the ability to call 7zip from the command line does have some use cases.

(Note that the [lzma module](https://docs.python.org/3/library/lzma.html) in the standard library may already be a solution to many problems and should be tried first)

First, let's try creating a sample 7z file using the command line. The *subprocess* module in the standard library is one way to call the 7zip program directly, as if it were on the command line. This code will write to a temporary directory in C:\Temp and assumes that you are using Windows. Please modify the temp_dir accordingly if you are using another operating system. Also, please modify the exact location of 7zip on your system using the loc_7z variable. Alternatively, if the 7zip location has been added to the system path, you may just use "7z" or "7z.exe".

In [1]:
import os
import subprocess

temp_dir = r"C:\Temp"
loc_7z = r"C:\Program Files\7-Zip\7z.exe"
# Where the final 7z file will be written
dummy_out_path = os.path.join(temp_dir, "dummy.7z")

# If this directory does not exist, it will need to be created
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    
# Create a dummy file that is roughly 100 kB in size
dummy_file_path = os.path.join(temp_dir, "dummy.txt")
with open(dummy_file_path, "w") as f:
    size = 100000 # in bytes
    f.write("1" * size)

# Now use the subprocess command to run 7zip. Warning- all paths should be 
# enclosed in quotation marks just in case there are any spaces in the path
# The "a" command given to 7z.exe means "add to archive"
archive_command = r'"{}" a "{}" "{}"'.format(loc_7z, dummy_out_path, dummy_file_path)
subprocess.call(archive_command, shell=True)

# Delete the original dummy file
os.remove(dummy_file_path)

The actual line of command-line code that was called using subprocess above is:

*"C:\Program Files\7-Zip\7z.exe" a "C:\Temp\dummy.7z" "C:\Temp\dummy.txt"*

unless any of the default variables were changed. The "a" command given to 7z.exe is used to make an archive, and the first argument afterwards is the output location of the archive, and the second argument is the location of the original file. Check the path "C:\Temp\dummy.7z" and you will see the file you just created. Note that this path will be different if you changed the temp_dir or dummy_out_path variables above.

There are two common problems that may be encountered here. First, the 7zip program may not always be added to the system PATH. It's up to you if you would like to do so, but if you don't know what modifying the PATH variable means, you should probably just specify the full path to the 7z program. This is usually something like "C:\Program Files\7-zip\7z.exe" on Windows, but you can check your default path by trying to re-install the 7zip program. Second, it appears that you may be able to only run one command-line version of 7zip at a time. If you are trying to run multiple command-line programs at once, or if you already have an unzip operation running, this code may not work on your system.

Now, let's extract the archive that was created above. 

In [2]:
# The "e" command given to 7z.exe means "extract from archive"
# WARNING- One of the most common mistakes is that there is NO space between the
# -o flag and the actual output directory (e.g. it should look like: "-oC:\Temp")
extract_command = r'"{}" x "{}" -o"{}"'.format(loc_7z, dummy_out_path, temp_dir)
subprocess.call(extract_command, shell=True);

The actual line of command-line code that was called using subprocess above is:

*"C:\Program Files\7-Zip\7z.exe" x "C:\Temp\dummy.txt" -o"C:\Temp"*

Important note- there is no space in between the -o and "C:\Temp"; this is not a typo. Take a look at the "C:\Temp" directory and you will notice that the dummy.txt file has been successfully extracted from the dummy.7z file!